In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import minmax_scale

In [2]:
# #PATH
path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"
path_to_predict = "F:/178/Tugas Akhir/dataset/predict/"

# path_info = "C:/Users/USER/Downloads/Tugas Akhir/dataset/annotations/"
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/coba fitur/"
# path_to_experiment ="C:/Users/USER/Downloads/Tugas Akhir/dataset/experiment/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"
# path_to_predict = "C:/Users/USER/Downloads/Tugas Akhir/dataset/predict/"

In [3]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

def split(data,value):
    
    train_data, test_data, train_values, test_values = train_test_split(data,value,test_size=0.3,random_state=1000)
    
    return train_data, test_data, train_values, test_values

def training(data,values,clf):
    trained = clf.fit(data,values)
    
    return trained
    
def predict(test,true,clf):
    pred = clf.predict(test)
    df = pd.DataFrame(data=[pred,true])
    
    return pred,df

def score(pred,true):
    r2 = r2_score(true, pred)
    
    return r2

def crosval(clf,data,values):
    scores = cross_val_score(clf,data,values,scoring='r2',cv=10)
    
    return scores

def save_model(model,case,types):
    path = path_to_model+types+case+'.sav' # save the model
    joblib.dump(model, path)

def save_predict(df1,df2,case):
    path = path_to_predict+case+".xlsx" # save the model
    arousal = df1.T
    valence = df2.T
    merged = arousal.merge(valence, how='outer', left_index=True, right_index=True)
    merged.to_excel(path,header=None,index=None)

In [4]:
kernels = 1.0 * RationalQuadratic(length_scale=1.0, alpha=1.0, 
                            length_scale_bounds=(1e-6, 1e6), 
                            alpha_bounds=(1e-6, 1e6))

# CASE 1

In [5]:
datafitur = "case1" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean,feature_range=(-1, 1))

#split arousal
train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar)
#split valence
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val)
print(train_data_a.shape)
print(test_data_a.shape)

(744, 26)
(520, 26)
(224, 26)


INITIAL MODEL

In [6]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, copy_X_train=False,  
                    random_state=1000) #init Arousal

In [7]:
print("arousal:\t",gpr_a.kernel)
print("arousal:\t",gpr_v.kernel)

arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)
arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [8]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [9]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [10]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 1.17**2 * RationalQuadratic(alpha=0.022, length_scale=4.78)
valence:	 0.5**2 * RationalQuadratic(alpha=0.21, length_scale=1.29)


AROUSAL

In [11]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.09,0.24,0.59,-0.41,-0.41,-0.09,-0.81,0.26,-0.44,-0.09,...,0.49,0.12,0.00,-0.16,-0.06,-0.00,0.32,0.36,-0.23,-0.45
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.64


VALENCE

In [12]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.16,0.32,0.47,-0.15,-0.07,0.10,-0.31,0.23,-0.08,0.05,...,0.42,0.34,0.01,-0.04,-0.01,0.05,0.16,-0.01,-0.13,-0.44
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence: 0.29


REGRESSION

In [13]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [14]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [15]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [16]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 2

In [17]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 8)


INITIAL MODEL

In [18]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,
                    random_state=1000) #init Valence

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,
                    random_state=1000) #init Valence

In [19]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)
valence:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [20]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [21]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [22]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 0.804**2 * RationalQuadratic(alpha=0.0193, length_scale=0.123)
valence:	 0.67**2 * RationalQuadratic(alpha=0.0301, length_scale=0.0858)


AROUSAL

In [23]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.09,-0.25,0.33,-0.45,-0.33,-0.23,-0.72,0.14,-0.23,0.07,...,0.36,0.05,-0.38,-0.17,0.23,-0.21,0.17,0.23,-0.4,-0.24
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.5,-0.82


R2 Score Arousal:	0.55


VALENCE

In [24]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.19,-0.14,0.31,-0.35,-0.05,-0.01,-0.34,0.26,0.20,0.21,...,0.36,0.41,-0.37,-0.06,0.21,0.10,0.18,0.18,-0.36,0.09
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.30


REGRESSION

In [25]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [26]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [27]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [28]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 3

In [29]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 3)


INITAL MODEL

In [30]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, 
                    random_state=1000) #init Valence

In [31]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)
valence:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [32]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [33]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [34]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 0.881**2 * RationalQuadratic(alpha=0.0107, length_scale=0.00403)
valence:	 0.495**2 * RationalQuadratic(alpha=0.0336, length_scale=3.13e-05)


AROUSAL

In [35]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.03,0.02,0.42,-0.37,-0.01,-0.17,-0.40,0.23,-0.36,-0.13,...,0.42,0.23,-0.34,-0.26,0.01,0.00,0.26,0.32,-0.46,-0.49
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.48


VALENCE

In [36]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.15,0.19,0.23,-0.18,0.00,0.07,-0.13,0.23,-0.18,0.13,...,0.23,0.23,-0.06,0.01,0.04,0.02,0.17,0.20,-0.20,-0.13
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.18


REGRESSION

In [37]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [38]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [39]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [40]:
save_predict(df_a,df_v,datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 4

In [41]:
datafitur =  "case4" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 24)


INITIAL MODEL

In [42]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Valence

In [43]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)
valence:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [44]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

PREDICT

In [45]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

SET PARAM

In [46]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 0.688**2 * RationalQuadratic(alpha=0.051, length_scale=0.858)
valence:	 0.407**2 * RationalQuadratic(alpha=0.451, length_scale=0.681)


AROUSAL

In [47]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.17,0.02,0.03,-0.30,-0.23,0.05,-0.28,0.13,-0.41,-0.20,...,0.15,0.24,0.08,-0.22,0.20,0.16,0.19,0.27,-0.43,-0.15
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.30


VALENCE

In [48]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.06,0.12,0.10,-0.02,0.09,0.19,-0.10,0.10,-0.09,0.03,...,0.04,-0.06,0.07,-0.11,0.10,0.10,0.09,0.10,-0.13,0.03
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.12


REGRESSION

In [49]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

AROUSAL

In [50]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [51]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [52]:
save_predict(df_a,df_v,datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 5

In [53]:
datafitur = "case5" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

#split arousal
train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar)
#split valence
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) 

(744, 34)


In [54]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

In [55]:
print("arousal:\t",gpr_a.kernel)
print("arousal:\t",gpr_v.kernel)

arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)
arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)


In [56]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [57]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [58]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 1.22**2 * RationalQuadratic(alpha=0.0193, length_scale=6.45)
valence:	 0.857**2 * RationalQuadratic(alpha=0.0427, length_scale=2.81)


In [59]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.02,0.17,0.48,-0.52,-0.38,-0.16,-0.84,0.27,-0.30,-0.08,...,0.47,0.11,-0.28,-0.29,-0.06,0.03,0.32,0.44,-0.23,-0.32
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.67


In [60]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.22,0.23,0.40,-0.40,-0.09,-0.01,-0.36,0.32,0.18,0.11,...,0.47,0.50,-0.28,-0.19,-0.05,0.08,0.17,0.12,-0.32,-0.14
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence: 0.37


In [61]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [62]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [63]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 6

In [64]:
datafitur = "case6" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape) 

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 29)


In [65]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, 
                    random_state=1000) #init Valence

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, 
                    random_state=1000) #init Valence

In [66]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)
valence:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)


In [67]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [68]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [69]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 1.16**2 * RationalQuadratic(alpha=0.0211, length_scale=4.98)
valence:	 0.502**2 * RationalQuadratic(alpha=0.204, length_scale=1.4)


In [70]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.07,0.22,0.60,-0.45,-0.38,-0.07,-0.81,0.29,-0.45,-0.10,...,0.50,0.11,0.01,-0.21,-0.13,-0.03,0.32,0.39,-0.23,-0.45
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.65


In [71]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.18,0.33,0.45,-0.20,-0.04,0.12,-0.31,0.28,-0.12,0.05,...,0.45,0.37,0.02,-0.07,-0.05,0.05,0.16,0.03,-0.16,-0.44
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.30


In [72]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [73]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [74]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [75]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 7

In [76]:
datafitur =  "case7" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 50)


In [77]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True, 
                    random_state=1000) #init Valence

In [78]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)
valence:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)


In [79]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [80]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [81]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 1.09**2 * RationalQuadratic(alpha=0.0355, length_scale=8.92)
valence:	 0.459**2 * RationalQuadratic(alpha=0.429, length_scale=2.58)


In [82]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.13,0.21,0.56,-0.49,-0.37,-0.15,-0.65,0.28,-0.32,0.02,...,0.50,0.1,-0.08,-0.28,-0.03,0.01,0.39,0.44,-0.29,-0.55
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.0,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.64


In [83]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.04,0.33,0.54,0.01,0.04,0.13,-0.26,0.21,-0.07,0.06,...,0.24,-0.04,0.03,-0.18,0.01,0.03,0.03,0.12,-0.16,-0.22
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.30


In [84]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [85]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [86]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [87]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 8

In [88]:
datafitur =  "case8" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 37)


In [89]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Valence

In [90]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)
valence:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)


In [91]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [92]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [93]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 1.2**2 * RationalQuadratic(alpha=0.0192, length_scale=6.44)
valence:	 0.865**2 * RationalQuadratic(alpha=0.0421, length_scale=2.98)


In [94]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.01,0.16,0.50,-0.53,-0.35,-0.16,-0.84,0.31,-0.33,-0.07,...,0.49,0.11,-0.25,-0.30,-0.11,0.01,0.34,0.44,-0.24,-0.32
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.68


In [95]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.22,0.22,0.39,-0.36,-0.08,-0.02,-0.36,0.37,0.12,0.12,...,0.48,0.50,-0.28,-0.19,-0.06,0.08,0.21,0.14,-0.32,-0.13
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.39


In [96]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [97]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [98]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [99]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 9

In [100]:
datafitur =  "case9" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 58)


In [101]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Valence

In [102]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)
valence:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)


In [103]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [104]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [105]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 1.11**2 * RationalQuadratic(alpha=0.0313, length_scale=10.3)
valence:	 0.837**2 * RationalQuadratic(alpha=0.0606, length_scale=5.2)


In [106]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.07,0.14,0.50,-0.57,-0.38,-0.23,-0.72,0.26,-0.22,-0.05,...,0.52,0.12,-0.24,-0.33,-0.04,0.03,0.36,0.46,-0.3,-0.45
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.5,-0.82


R2 Score Arousal:	0.67


In [107]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.15,0.30,0.55,-0.17,-0.02,-0.02,-0.39,0.29,0.17,0.04,...,0.31,0.09,-0.23,-0.25,-0.10,0.02,0.03,0.15,-0.28,-0.03
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.38


In [108]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [109]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [110]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [111]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence

# CASE 10

In [112]:
datafitur =  "case10" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

scaled_ar = minmax_scale(arousal_mean.values,feature_range=(-1, 1))
scaled_val = minmax_scale(valence_mean.values,feature_range=(-1, 1))

train_data_a, test_data_a, train_values_a, test_values_a = split(data,scaled_ar) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,scaled_val) #split valence

(744, 61)


In [113]:
gpr_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Arousal

gpr_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=10, 
                    normalize_y=True,  
                    random_state=1000) #init Valence

In [114]:
print("arousal:\t",gpr_a.kernel)
print("valence:\t",gpr_v.kernel)

arousal:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)
valence:	 1**2 * RationalQuadratic(alpha=1, length_scale=1)


In [115]:
trained_a = training(train_data_a,train_values_a,gpr_a)
trained_v = training(train_data_v,train_values_v,gpr_v)

In [116]:
# #Predict Arousal
pred_a,df_a = predict(test_data_a, test_values_a, trained_a)
# #Predict Valence
pred_v,df_v = predict(test_data_v, test_values_v, trained_v)

r2_a = score(pred_a,test_values_a) #Score Arousal
r2_v = score(pred_v,test_values_v) #Score Valence

In [117]:
trained_a.set_params(kernel=trained_a.kernel_)
trained_v.set_params(kernel=trained_v.kernel_)

print("arousal:\t",trained_a.kernel)
print("valence:\t",trained_v.kernel)

arousal:	 1.07**2 * RationalQuadratic(alpha=0.0319, length_scale=10.1)
valence:	 0.849**2 * RationalQuadratic(alpha=0.0573, length_scale=5.28)


In [118]:
df_a = df_a.round(2)

display(df_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.07,0.16,0.53,-0.60,-0.32,-0.24,-0.71,0.30,-0.28,-0.03,...,0.55,0.13,-0.21,-0.33,-0.08,0.05,0.38,0.48,-0.32,-0.50
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.68


In [119]:
df_v = df_v.round(2)

display(df_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.15,0.29,0.55,-0.17,0.00,-0.02,-0.39,0.34,0.10,0.06,...,0.30,0.11,-0.23,-0.25,-0.11,0.03,0.04,0.17,-0.29,-0.04
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.38


In [120]:
# scores_a = crosval(gpr_a,data,scaled_ar) #Arousal Regression
# scores_v = crosval(gpr_v,data,scaled_val) #Arousal Regression

In [121]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

In [122]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [123]:
save_predict(df_a.round(2),df_v.round(2),datafitur)
save_model(trained_a,datafitur,'arousal/') #save model Arousal
save_model(trained_v,datafitur,'valence/') #save model Valence